### Khai báo thư viện

In [23]:
import pandas as pd
import re

### Tải dữ liệu

In [24]:
data = pd.read_excel('../Dataset/Nha_Thuoc.xlsx')
data.head(5)

,LOAIHINHHN,PHAMVIHN,TENCOSO,DUONG,PHUONG,QUAN,CVXLCCHN,SOCCHN,NGCAPCCHN,NOICAPCCHN,SODDK,SOGCN,GPP_GDP_GSP,NGAYCAPGCN,NGAYHHGCN
0,Đại lý bán thuốc của doanh nghiệp,Bán lẻ thuốc thành phẩm thuộc danh mục thuốc t...,Đại lý số 24 - Công ty TNHH một thành viên Dượ...,"C5/22, Đường Hưng Nhơn, Ấp 3",Xã Tân Kiên,Huyện Bình Chánh,Ds Trương Việt Cường,01531/HCM-CCHND,20/02/2014,Sở Y tế TP.HCM,6427/HCM-ĐKKDD,NaN,NaN,NaN,NaN
1,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Hộ kinh doanh Nhà thuốc Kim Châu,"9, Đường số 5, Khu phố 2",Phường Linh Xuân,Quận Thủ Đức,Ds. Trương Việt Cường,4524/CCHN-D-SYT-HCM,14/06/2019,Sở Y tế TP.HCM,7969/ĐKKDD-HCM,9807.0,GPP,18/03/2020,NaN
2,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Nhà thuốc Phương Hiền,"493, Man Thiện, Khu phố 5",Phường Tăng Nhơn Phú A,Quận 9,NaN,2306/ĐNAI-CCHND,13/07/2016,Sở Y tế Tỉnh Đồng Nai,1660/ĐKKDD-HCM,7816.0,GPP,01/12/2017,01/12/2020
3,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Địa điểm kinh doanh Công ty cổ phần dược phẩm ...,"1097B, Tỉnh lộ 43, Khu phố 2",Phường Bình Chiểu,Quận Thủ Đức,Ds. Trần Minh Trí,1131/CCHN-D-SYT-HCM,26/10/2017,Sở Y tế TP.HCM,8433/ĐKKDD-HCM,10113.0,GPP,10/07/2020,10/07/2023
4,Nhà thuốc,Bán lẻ thuốc thành phẩm,Nhà thuốc Nguyễn Nhi,"1B, Nguyễn Thượng Hiền",Phường 05,Quận Bình Thạnh,NaN,03135/HCM-CCHND,17/06/2015,Sở Y tế TP.HCM,5194/HCM-ĐKKDD,6121.0,GPP,13/10/2016,13/10/2019


### Kiểm tra dữ liệu bị thiếu

In [25]:
data.isna().sum()

LOAIHINHHN      114
PHAMVIHN         63
TENCOSO           0
DUONG             0
PHUONG           40
QUAN              2
CVXLCCHN       3775
SOCCHN            0
NGCAPCCHN       221
NOICAPCCHN        1
SODDK             2
SOGCN           735
GPP_GDP_GSP     735
NGAYCAPGCN     4606
NGAYHHGCN      4927
dtype: int64

Ta nhận thấy tồn tại dữ liệu bị thiếu ở các cột trong bảng dữ liệu, các cột `CVXLCCHN`, `SOGCN`, `GPP_GDP_GSP`, `NGAYCAPGCN`, `NGAYHHGCN` đều là các thuộc tính đặc thù, không quá quan trọng khi phân tích các thuộc tính đặc trưng khác nên tạm thời ta sẽ giữ lại các giá trị bị thiếu ở cột này mà chỉ đi xử lí ở các cột `LOAIHINHHN`, `PHAMVIHN`, `QUAN`

In [26]:
data = data.dropna(subset=['LOAIHINHHN','PHAMVIHN','QUAN'])
data.isna().sum()

LOAIHINHHN        0
PHAMVIHN          0
TENCOSO           0
DUONG             0
PHUONG           33
QUAN              0
CVXLCCHN       3758
SOCCHN            0
NGCAPCCHN       221
NOICAPCCHN        1
SODDK             2
SOGCN           729
GPP_GDP_GSP     729
NGAYCAPGCN     4599
NGAYHHGCN      4919
dtype: int64

In [27]:
data.shape

(16508, 15)

### Kiểm tra dữ liệu trùng lắp

In [28]:
print('Số lượng dòng dữ liệu trùng lắp:', data.duplicated().sum())

Số lượng dòng dữ liệu trùng lắp: 84


Ta nhận thấy có 84 dòng dữ liệu bị trùng lắp, ta tiến hành loại bỏ các dòng dữ liệu này

In [29]:
data.drop_duplicates(inplace=True)
print('Số lượng dòng dữ liệu sau khi loại trùng:', data.duplicated().sum())

Số lượng dòng dữ liệu sau khi loại trùng: 0


### Gom các quận trong thành phố Thủ Đức
Sau khi đã xử lí trùng, ta gom các quận Thủ Đức, 2 và 9 lại thành thành phố Thủ Đức.

In [30]:
districts = ['Quận Thủ Đức', 'Quận 2', 'Quận 9']

def normalize_address(address):
    if address in districts:
        return "TP. Thủ Đức"
    return address

data['QUAN'] = data['QUAN'].apply(normalize_address)

data['QUAN'].unique()

array(['Huyện Bình Chánh', 'TP. Thủ Đức', 'Quận Bình Thạnh', 'Quận 7',
       'Quận Gò Vấp', 'Quận 10', 'Huyện Hóc Môn', 'Huyện Củ Chi',
       'Quận 8', 'Quận 12', 'Quận Tân Phú', 'Huyện Cần Giờ', 'Quận 1',
       'Quận 11', 'Quận Bình Tân', 'Quận 4', 'Quận 5', 'Quận Tân Bình',
       'Quận 3', 'Huyện Nhà Bè', 'Quận 6', 'Quận Phú Nhuận'], dtype=object)

### Tinh chỉnh định dạng các quận

Tinh chỉnh các quận về giống định dạng trong tập tin liên quan đến cơ sở nhà thuốc để dễ dàng trực quan.

In [31]:
# Hàm giữ nguyên các quận có số, xóa "Quận" ở các quận không có số
def remove_quan_name(quan):
    if "Quận " in quan and not quan.split()[1].isdigit():
        return quan.replace("Quận ", "")
    return quan

# Áp dụng hàm trên cột QUAN
data['QUAN'] = data['QUAN'].apply(remove_quan_name)
data['QUAN'] = data['QUAN'].str.replace('Huyện ', '', regex=False)
data['QUAN'].unique()

array(['Bình Chánh', 'TP. Thủ Đức', 'Bình Thạnh', 'Quận 7', 'Gò Vấp',
       'Quận 10', 'Hóc Môn', 'Củ Chi', 'Quận 8', 'Quận 12', 'Tân Phú',
       'Cần Giờ', 'Quận 1', 'Quận 11', 'Bình Tân', 'Quận 4', 'Quận 5',
       'Tân Bình', 'Quận 3', 'Nhà Bè', 'Quận 6', 'Phú Nhuận'],
      dtype=object)

### Làm gọn loại hình nhà thuốc

In [32]:
data.sample(5)

,LOAIHINHHN,PHAMVIHN,TENCOSO,DUONG,PHUONG,QUAN,CVXLCCHN,SOCCHN,NGCAPCCHN,NOICAPCCHN,SODDK,SOGCN,GPP_GDP_GSP,NGAYCAPGCN,NGAYHHGCN
13364,Cơ sở bán lẻ thuốc: Nhà thuốc,Bán lẻ thuốc trừ vắc xin.,Nhà thuốc Ngọc Hạnh,"570, Tỉnh lộ 43, Khu phố 3",Phường Tam Bình,TP. Thủ Đức,Ds Trần Minh Trí,03733/HCM-CCHND,24/02/2015,Sở Y tế TP.HCM,1614/HCM-ĐKKDD,4681.0,GPP,01/07/2017,01/07/2020
8198,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Hộ kinh doanh nhà thuốc tư nhân Hoa Lợi Đ.18,"58, Đường 18, khu phố 5",Phường Linh Trung,TP. Thủ Đức,NaN,7553/CCHN-D-SYT-ĐNAI,10/04/2023,Sở Y tế Tỉnh Đồng Nai,14959/ĐKKDD-HCM,15311.0,GPP,NaN,NaN
11193,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Nhà thuốc Ngọc Minh,"41/9B, Huỳnh Tấn Phát, Khu phố 1",Phường Tân Thuận Tây,Quận 7,Ds Bùi Thanh Nguyệt,00760/HCM-CCHND,28/08/2013,Sở Y tế TP.HCM,3806/ĐKKDD-HCM,2332.0,GPP,05/10/2018,05/10/2021
16149,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Công ty TNHH Dr. Huy Clinic,"Số 07, Đường số 01, Khu dân cư Cityland",Phường 10,Gò Vấp,Ds. Nguyễn Thị Ngọc Diễm,5894/CCHN-D-SYT-HCM,22/05/2020,Sở Y tế TP.HCM,10046/ĐKKDD-HCM,11162.0,GPP,31/05/2021,31/05/2024
6622,Cơ sở bán buôn thuốc,"Bán buôn thuốc hóa dược, thuốc dược liệu, thuố...",Công ty TNHH Dược phẩm Tuyết Hải,"195C, Nguyễn Chí Thanh",Phường 12,Quận 5,Ds Lê Thị Thu Hòa,2080/HCM-CCHND,20/05/2014,Sở Y tế TP.HCM,9181/ĐKKDD-HCM,589.0,GDP,17/11/2023,17/11/2026


#### Kiểm tra các giá trị khác biệt trong cột `LOAIHINHHN`

In [33]:
data['LOAIHINHHN'].unique()

array(['Đại lý bán thuốc của doanh nghiệp',
       'Cơ sở bán lẻ thuốc: Nhà thuốc', 'Nhà thuốc',
       'Cơ sở bán buôn thuốc',
       'Cơ sở sản xuất thuốc đông y, thuốc từ dược liệu',
       'Doanh nghiệp bán buôn thuốc', 'Nhà thuốc bệnh viện công lập',
       'Cơ sở bán lẻ thuốc: Cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền',
       'Cơ sở bán lẻ thuốc: Nhà thuốc.', 'Cơ sở bán buôn thuốc.',
       'Doanh nghiệp xuất khẩu, nhập khẩu thuốc',
       'Cơ sở xuất nhập khẩu thuốc',
       'Doanh nghiệp sản xuất, bán buôn thuốc',
       'Cơ sở bán lẻ thuốc đông y, thuốc từ dược liệu',
       'Cơ sở bán lẻ thuốc: Nhà thuốc. Cơ sở bán lẻ thuốc: Cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền.',
       'Nhà thuốc của Doanh nghiệp', 'Cơ sở bán lẻ thuốc: Kệ thuốc',
       'Cơ sở bán buôn dược liệu, thuốc đông y, thuốc từ dược liệu',
       'Cơ sở bán buôn dược liệu',
       'Cơ sở bán buôn thuốc, nguyên liệu làm thuốc',
       'Doanh nghiệp sản xuất thuốc dùng

Ta nhận thấy dòng chữ "Cơ sở bán lẻ thuốc" là khá dư thừa trong chuỗi, ta tiến hành loại bỏ chuỗi này. Đồng thời có dấu '.' thừa ở cuối câu ta cũng sẽ loại bỏ

In [34]:
data['LOAIHINHHN'] = data['LOAIHINHHN'].str.replace('Cơ sở bán lẻ thuốc: ', '', regex=True)
data['LOAIHINHHN'] = data['LOAIHINHHN'].apply(lambda x: x.rstrip('.') if x.endswith('.') else x)
data['LOAIHINHHN'] = data['LOAIHINHHN'].str.lower()

#### Kiểm tra lại sau khi đã loại bỏ

In [35]:
unique_counts = data['LOAIHINHHN'].value_counts()

for value, count in unique_counts.items():
    print(f"'{value}': {count} lần")

'nhà thuốc': 13429 lần
'cơ sở bán buôn thuốc': 1976 lần
'cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền': 365 lần
'đại lý bán thuốc của doanh nghiệp': 152 lần
'doanh nghiệp bán buôn thuốc': 150 lần
'cơ sở bán lẻ thuốc đông y, thuốc từ dược liệu': 72 lần
'nhà thuốc của doanh nghiệp': 52 lần
'cơ sở bán buôn thuốc, nguyên liệu làm thuốc': 49 lần
'doanh nghiệp xuất khẩu, nhập khẩu thuốc': 39 lần
'cơ sở sản xuất thuốc đông y, thuốc từ dược liệu': 34 lần
'cơ sở bán buôn nguyên liệu làm thuốc': 21 lần
'cơ sở bán buôn dược liệu, thuốc đông y, thuốc từ dược liệu': 18 lần
'cơ sở chuyên bán lẻ thuốc dược liệu, thuốc cổ truyền': 14 lần
'nhà thuốc. cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền': 8 lần
'nhà thuốc bệnh viện công lập': 7 lần
'cơ sở chuyên bán lẻ dược liệu': 7 lần
'cơ sở bán buôn dược liệu': 5 lần
'kệ thuốc': 3 lần
'doanh nghiệp bán buôn dược liệu, thuốc đông y, thuốc từ dược liệu': 3 lần
'doanh nghiệp kinh doanh thuốc': 3 lần
'cơ sở xuất nhập khẩu th

In [36]:
data.isna().sum()

LOAIHINHHN        0
PHAMVIHN          0
TENCOSO           0
DUONG             0
PHUONG           33
QUAN              0
CVXLCCHN       3745
SOCCHN            0
NGCAPCCHN       221
NOICAPCCHN        1
SODDK             2
SOGCN           723
GPP_GDP_GSP     723
NGAYCAPGCN     4545
NGAYHHGCN      4869
dtype: int64

### Kiểm tra các giá trị trong cột `PHAMVIHN`

In [37]:
print(data['PHAMVIHN'].unique())
print('\nSố lượng giá trị khác biệt trong cột:',len(data['PHAMVIHN'].unique()))

['Bán lẻ thuốc thành phẩm thuộc danh mục thuốc thiết yếu (trừ thuốc gây nghiện, thuốc hướng tâm thần và tiền chất dùng làm thuốc và các thuốc kê đơn).'
 'Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm bảo quản điều kiện thường (Có bao gồm thuốc phải kiểm soát đặc biệt: Thuốc dạng phối hợp có chứa dược chất gây nghiện; Thuốc dạng phối hợp có chứa dược chất hướng thần; Thuốc dạng phối hợp có chứa tiền chất; Thuốc độc; Thuốc trong danh mục thuốc, dược chất thuộc danh mục chất bị cấm sử dụng trong một số ngành, lĩnh vực).'
 'Bán lẻ thuốc thành phẩm'
 'Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm (Có bao gồm thuốc phải kiểm soát đặc biệt: Thuốc dạng phối hợp có chứa dược chất gây nghiện; Thuốc dạng phối hợp có chứa dược chất hướng thần; thuốc dạng phối hợp có chứa tiền chất; Thuốc độc; Thuốc trong danh mục thuốc, dược chất thuộc danh mục chất bị cấm sử dụng trong một số ngành, lĩnh vực).'
 'Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, si

Với số lượng khác biệt trong cột quá lớn, ta tạm không xét các mô tả chi tiết trong dấu ngoặc tròn để dễ dàng hơn trong phân tích. Đồng thời, các thông tin 'bán buôn' và 'bán lẻ' khá giống nhau, ta sẽ thay thế bằng 'bán' và loại bỏ các từ không quá quan trọng.

In [38]:
def remove_parentheses(text):
    parts = re.split(r"\(", text)
    if parts:
        return parts[0]
    return text

#Lọc các từ trong dấu ngoặc tròn
data['PHAMVIHN'] = data['PHAMVIHN'].apply(remove_parentheses)
data['PHAMVIHN'] = data['PHAMVIHN'].str.lower()

#Lọc các từ không mang nhiều ý nghĩa
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bán lẻ', 'bán')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bán buôn', 'bán')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bảo quản ở điều kiện thường ', '')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bảo quản điều kiện thường ', '')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace(' và', ',')
data['PHAMVIHN'] = data['PHAMVIHN'].apply(lambda x: x.rstrip('. ').rstrip() if 
                                         x.endswith('.') or x.endswith(' ') else x)

In [39]:
unique_counts = data['PHAMVIHN'].value_counts()
print(len(unique_counts))
for value, count in unique_counts.items():
    print(f"'{value}': {count} lần")

110
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm': 13914 lần
'bán thuốc thành phẩm': 980 lần
'bán thuốc trừ vắc xin': 480 lần
'bán thuốc dược liệu, thuốc cổ truyền': 201 lần
'bán thuốc thành phẩm thuộc danh mục thuốc thiết yếu': 152 lần
'bán dược liệu, thuốc dược liệu, thuốc cổ truyền': 132 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền': 117 lần
'bán thuốc đông y, thuốc từ dược liệu': 67 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, vắc xin, sinh phẩm': 47 lần
'bán dược liệu': 41 lần
'sản xuất thuốc đông y, thuốc từ dược liệu': 26 lần
'bán thuốc hóa dược': 21 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm bảo quản điều kiện thường': 19 lần
'bán dược liệu, vị thuốc cổ truyền': 13 lần
'xuất khẩu, nhập khẩu thuốc thành phẩm, vắc xin, sinh phẩm y tế': 12 lần
'xuất khẩu, nhập khẩu thuốc thành phẩm': 12 lần
'bán thuốc hóa dược, sinh phẩm': 10 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm bảo quản ở điều

### Loại bỏ các cột không mang nhiều ý nghĩa

Ta nhận thấy các cột `TENCOSO`, `SOCCHN`, `SODDK`, `SOGCN` là các cột không mang lại nhiều thông tin đáng giá để phân tích, ta tiến hành loại các cột này

In [40]:
data = data.drop(['TENCOSO', 'SOCCHN', 'SODDK', 'SOGCN'], axis=1)

Bộ dữ liệu sau khi đã tiền xử lí xong có dạng:

In [41]:
data.sample(7)

,LOAIHINHHN,PHAMVIHN,DUONG,PHUONG,QUAN,CVXLCCHN,NGCAPCCHN,NOICAPCCHN,GPP_GDP_GSP,NGAYCAPGCN,NGAYHHGCN
1644,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","11, Nguyễn Quý Anh",Phường Tân Sơn Nhì,Tân Phú,Ds. Nguyễn Thị Ngọc Diễm,19/11/2020,Sở Y tế TP.HCM,GPP,15/03/2021,NaN
9983,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","152, Trần Bá Giao",Phường 05,Gò Vấp,Ds. Trần Minh Trí,18/01/2023,Sở Y tế TP.HCM,GPP,02/06/2023,02/06/2026
12488,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","4 TMT 2A, Khu phố 5",Phường Trung Mỹ Tây,Quận 12,Ds. Bùi Hùng Cường,18/08/2023,Sở Y tế TP.HCM,GPP,03/11/2023,03/11/2026
6563,cơ sở bán buôn thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","108/1, Ngô Quyền",Phường 05,Quận 10,Ds. Bùi Thanh Nguyệt,12/10/2020,Sở Y tế TP.HCM,GDP,NaN,NaN
12371,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","36, Lý Phục Man",Phường Bình Thuận,Quận 7,Ds. Trần Thị Như Thủy,18/10/2017,Sở Y tế TP.HCM,GPP,NaN,NaN
2551,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","27D, Bình Phú",Phường 10,Quận 6,Ds Trần Thị Như Thủy,15/01/2014,Sở Y tế TP.HCM,GPP,NaN,NaN
2470,"cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu...","bán dược liệu, thuốc dược liệu, thuốc cổ truyền","số 86A, Hải Thượng Lãn Ông",Phường 10,Quận 5,NaN,29/12/2015,Sở Y tế Tỉnh Long An,NaN,NaN,NaN


### Lưu lại bộ dữ liệu

#### Lưu dưới dạng csv

In [42]:
data.to_csv('../Dataset/NhaThuoc_DaXuLy.csv', header=True, index=False)

#### Lưu dưới dạng excel

In [43]:
data.to_excel('../Dataset/NhaThuoc_DaXuLy.xlsx', header=True, index=False)